Import necessary modules

In [1]:
import numpy as np 
import pandas as pd
import urllib.request 
import requests, io
import urllib, re, string
import datetime

Include for nicer format of tables when printing

In [2]:
pd.set_option('display.notebook_repr_html', True)

def _repr_latex_(self):
    return "\centering{%s}" % self.to_latex()

pd.DataFrame._repr_latex_ = _repr_latex_  # monkey patch pandas DataFrame

### Data Source and Date of Retrieval

Data Retrieved 1/23/2017  

https://urldefense.proofpoint.com/v2/url?u=http-3A__mint.bio.uniroma2.it_mitab_MINT-5FMiTab.txt&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=Bv-GMiHdQIC4fRHi4JERcK71UMdrk1EvTGoRMpquPiM&e=  (Mint) 

Data Retrieved 1/25/2017  

https://urldefense.proofpoint.com/v2/url?u=https-3A__thebiogrid.org_downloads_archives_Release-2520Archive_BIOGRID-2D3.4.144_BIOGRID-2DALL-2D3.4.144.mitab.zip&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=mI2tEi0k6-2vBnUAJ8GD2-TCPVdPWzpMbje_637goZs&e=  (BioGrid)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__www.innatedb.com_download_interactions_innatedb-5Fppi.mitab.gz&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=M4I-F00djsSBy83R9CVNADb-OMwpiel94VVNTR-murY&e=  (InnateDB)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__dip.doe-2Dmbi.ucla.edu_dip_File.cgi-3FFN-3D2016_tab25_dip20160731.txt&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=D7qKgclA4IbvfDjbxccL8LSG3UsPa7AU4ant8b8qXWo&e=  (DIP)

Data Retrieved 1/26/2017   
https://urldefense.proofpoint.com/v2/url?u=http-3A__mentha.uniroma2.it_doDownload.php-3Ffile-3Dorganisms_9606.zip&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=RMnXZik8FZh0xdEL4OZPLacO9h0pmWzSKsHvkFp3Jt4&e=  (mentha)  
https://urldefense.proofpoint.com/v2/url?u=http-3A__www.ebi.ac.uk_intact_downloads&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=F7QsaA5RcDLVV_g_n7uDXIrPdh-SJk3JYZjYAE8HzO4&e=  (IntAct)  

Data Retrieved 2/1/2017     
http://amp.pharm.mssm.edu/X2K/datasets/SNAVI.sig (SAVI)  
http://amp.pharm.mssm.edu/X2K/datasets/KEA.zip (KEA)  
http://amp.pharm.mssm.edu/X2K/datasets/ppid.sig (ppid)  

In [412]:
def concatliketerms(df):
    
    df.sort_values(by=['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)    
    df.reset_index(inplace=True)
    df.drop('index', axis=1, inplace=True)
    oldSize = df.shape
    newSize = 0
    j = 1
    while oldSize != newSize:
        df.reset_index(inplace=True)
        df.drop('index', axis=1, inplace=True)
        df.sort_values(by=['Protein A (gene name)', 'Protein B (gene name)'], inplace=True) 
        # concattonate like terms and remove duplicates
        i = 1
        end = len(df.index)-1
        for index in df.index:
            if i % 1000 == 0:
                print('\r', 'Loop '+ str(j)+': ' +str(i)+' Out of '+ str(len(df.index)), end=' ', flush=True)
            if index in df.index.values and index != end:
                if df.ix[index, 'Protein A (gene name)'] == df.ix[index+1, 'Protein A (gene name)']:
                    if df.ix[index, 'Protein B (gene name)'] == df.ix[index+1, 'Protein B (gene name)']:
                        df.ix[index, 'PubMed ID'] = str(df.ix[index, 'PubMed ID']) +'|'+str(df.ix[index+1, 'PubMed ID'])
                        df.ix[index, 'Source databases'] = df.ix[index, 'Source databases'] +'|'+df.ix[index+1, 'Source databases']
                        df.drop(index+1, axis=0, inplace=True)
                        
            i += 1
        j += 1
        oldSize = newSize
        newSize = df.shape

In [3]:
# column identifiers for initial data (as taken from respective metadata)
col_name = ['Unique identifier for interactor A',
      'Unique identifier for interactor B',
      'Alternative identifier for interactor A',
      'Alternative identifier for interactor B',
      'Protein A (gene name)',
      'Protein B (gene name)',
      'Interaction detection methods',
      'First author',
      'PubMed ID',
      'NCBI Taxonomy identifier for interactor A',
      'NCBI Taxonomy identifier for interactor B',
      'Interaction types',
      'Source databases', 
      'Interaction identifier(s)',
      'Confidence score']

#### Read mint data from url and creat file

In [4]:
##read in data from Mint and write to file (only needs to be proformed once)
# content=urllib.request.urlopen('https://urldefense.proofpoint.com/v2/url?u=http-3A__mint.bio.uniroma2.it_mitab_MINT-5FMiTab.txt&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=Bv-GMiHdQIC4fRHi4JERcK71UMdrk1EvTGoRMpquPiM&e= ') 

# target = open('mint.tsv', 'wb')
# for line in content:
#     target.write(line)
# target.close()

## Mint Data

In [4]:
mint_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/mint.tsv', sep='\t', index_col=False, header=None)

In [6]:
# assign names to columns from meta data
for i in np.arange(len(col_name)):
    mint_df.rename(columns={i:col_name[i]}, inplace=True)

In [158]:
# get only human (or mouse) data (first protein)
n = mint_df['Protein A (gene name)'].values
b = [i for i,item in enumerate(n) if "human" in item]
mint_df = mint_df.ix[b]

In [159]:
mint_df =  mint_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data (second protein)
n = mint_df['Protein B (gene name)'].values
b = [i for i,item in enumerate(n) if "human" in item]
mint_df = mint_df.ix[b]

In [160]:
# select only relevent data
mint_df = mint_df[['Protein A (gene name)', 'Protein B (gene name)', 'PubMed ID', 'Source databases']]

In [161]:
# change column one to just show gene name
#i = 1
lst = []
for name in mint_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
mint_df['Protein A (gene name)'] = lst

In [162]:
# change column two to just show gene name
#i = 1
lst = []
for name in mint_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
mint_df['Protein B (gene name)'] = lst

In [163]:
# drop data for which no gene name is provited (or ensamble ID)
mint_df.dropna(how='any', inplace=True, axis=0)

In [164]:
mint_df.drop_duplicates(inplace=True)

In [165]:
# Get Only PubMed ID for publication identifier
lst = []

for pub in mint_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')[1].split(':')[1]
    lst.append(pub)
mint_df['PubMed ID'] = lst

In [166]:
mint_uf_df = mint_df.copy()

In [383]:
concatliketerms(mint_uf_df)

 Loop 4: 19000 Out of 19073 

In [170]:
mint_uf_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A1BG,GRB7,MINT-8089645,"psi-mi:""MI:0471""(MINT)"
1,A1BG,SMN1,MINT-8089645,"psi-mi:""MI:0471""(MINT)"
2,A1BG,CDKN1A,MINT-8089645,"psi-mi:""MI:0471""(MINT)"
3,A1BG,TK1,MINT-8089645,"psi-mi:""MI:0471""(MINT)"
28,A2M,ATP6V1B2,17174955,"psi-mi:""MI:0471""(MINT)"


In [384]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False) 

In [385]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [174]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
lst = []
for pub in mint_df['PubMed ID'].values:
    print('\r', str(i)+' Out of '+ str(len(mint_df['PubMed ID'].values)), end=' ', flush=True)
    lst.append(mint_df[mint_df['PubMed ID'] == pub].shape[0] <= 10)
    i +=1

mint_df = mint_df[lst]

 20133 Out of 20133 

In [175]:
# Drop data for which there is no PubMed ID
lst = []
for value in mint_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

mint_df = mint_df.drop(mint_df[lst].index)

In [176]:
# Drop any data missing information
mint_df.dropna(inplace=True)

In [380]:
concatliketerms(mint_df)

 Loop 4: 6000 Out of 6394 

In [180]:
mint_df.shape

(6492, 4)

In [381]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_human_filtered_%s.tsv'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False) 

In [382]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_human_filtered_%s.tsv.gz'% strhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28datetime.date.today&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=0AXIrr2R1Ws2uctDrGSv2BrKSII9GfdeCfYYet-im-Y&e= ()).replace('-', '_')
mint_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## BioGrid Data

In [21]:
biogrid_df =https://urldefense.proofpoint.com/v2/url?u=http-3A__pd.re&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=kUtB02_d29l-AX62DWgGfOrrMBgeV15cPbw0hAtCi7Y&e= ad_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/BIOGRID-ALL-3.4.144.mitab.txt', sep='\t', index_col=False)

In [121]:
# get only human (or mouse) data
n = biogrid_df['Taxid Interactor A'].values
b = [i for i,item in enumerate(n) if "taxid:9606" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [122]:
biogrid_df =  biogrid_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = biogrid_df['Taxid Interactor B'].values
b = [i for i,item in enumerate(n) if "taxid:9606" in item] #taxid:9606 for human, taxid:10090 for mouse
biogrid_df = biogrid_df.ix[b]

In [123]:
# select only relevent data
biogrid_df = biogrid_df[['Alt IDs Interactor A', 'Alt IDs Interactor B', 'Publication Identifiers', 'Source Database']]

In [124]:
# rename columns for consistency
biogrid_df.rename(columns={'Alt IDs Interactor A': 'Protein A (gene name)', 'Alt IDs Interactor B':'Protein B (gene name)', 'Publication Identifiers':'PubMed ID', 'Source Database':'Source databases'}, inplace=True)

In [125]:
# change column one to just show gene name
i = 1
lst = []
for name in biogrid_df['Protein A (gene name)']:
    if i % 100 == 0:
        print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[1].split(':')[1])
    else:
        lst.appendhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28np.na&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=7WM6dIU5B2DmwOyR_-PTnzSdk60Ct63WnSExIongu-A&e= n)
    i += 1
biogrid_df['Protein A (gene name)'] = lst

 303500 

In [126]:
# change column two to just show gene name
i = 1
lst = []
for name in biogrid_df['Protein B (gene name)']:
    if i % 100 == 0:
        print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[1].split(':')[1])
    else:
        lst.appendhttps://urldefense.proofpoint.com/v2/url?u=http-3A__-28np.na&d=DwIGAg&c=shNJtf5dKgNcPZ6Yh64b-A&r=FzzxHcpcdpeEl2SeS4RngXJfR5R4cEV8N2xULwaH8LE&m=H47AkOYhsWJidY3BTdZQzbIarlSka87imDJ8Qn9e6GY&s=7WM6dIU5B2DmwOyR_-PTnzSdk60Ct63WnSExIongu-A&e= n)
    i += 1
biogrid_df['Protein B (gene name)'] = lst

 303500 

In [127]:
biogrid_df.shape

(303566, 4)

In [128]:
# drop data for which no gene name is provited (or ensamble ID)
biogrid_df.dropna(how='any', inplace=True, axis=0)

In [129]:
biogrid_df.shape

(303566, 4)

In [130]:
# Get Only PubMed ID for publication identifier
lst = []

for pub in biogrid_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split(':')[1]
    lst.append(pub)
biogrid_df['PubMed ID'] = lst

In [131]:
#biogrid_df.drop_duplicates(['Aliases for A', 'Aliases for B'], inplace=True)
biogrid_df.drop_duplicates(inplace=True)

In [133]:
# uf = unfiltered
biogrid_uf_df = biogrid_df.copy()

In [137]:
concatliketerms(biogrid_uf_df)

 Loop 5: 281100 Out of 281178 

In [139]:
biogrid_uf_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A1BG,ABCC6,21988832,"psi-mi:""MI:0463""(biogrid)"
1,A1BG,SETD7,21988832,"psi-mi:""MI:0463""(biogrid)"
2,A1BG,PRDX4,21988832,"psi-mi:""MI:0463""(biogrid)"
3,A1CF,APOBEC1,10669759|11134005,"psi-mi:""MI:0463""(biogrid)|psi-mi:""MI:0463""(bio..."
4,A1CF,SYNCRIP,11134005,"psi-mi:""MI:0463""(biogrid)"


In [140]:
biogrid_df.shape

(284266, 4)

In [141]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
biogrid_uf_df.to_csv(filename, sep='\t', index=False) 

In [142]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
biogrid_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [144]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in biogrid_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(biogrid_df['PubMed ID'].unique())), end=' ', flush=True)
    if biogrid_df[biogrid_df['PubMed ID'] == pub].shape[0] > 10:
        biogrid_df.drop(biogrid_df[biogrid_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


 24500 Out of 22985 

In [145]:
biogrid_df.shape

(63817, 4)

In [147]:
lst = []
for value in biogrid_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

biogrid_df = biogrid_df.drop(biogrid_df[lst].index)

In [148]:
biogrid_df.dropna(inplace=True)

In [149]:
biogrid_df.shape

(63817, 4)

In [150]:
concatliketerms(biogrid_df)

 Loop 5: 61300 Out of 61387 

In [151]:
biogrid_df.shape

(61387, 4)

In [152]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
biogrid_df.to_csv(filename, sep='\t', index=False) 

In [153]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
biogrid_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## InnateDB Data

In [24]:
innatedb_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/innatedb_ppi.mitab', sep='\t', index_col=False)

In [182]:
# get only human (or mouse) data
n = innatedb_df['ncbi_taxid_A'].values
b = [i for i,item in enumerate(n) if "Human" in item]
innatedb_df = innatedb_df.ix[b]

In [183]:
innatedb_df =  innatedb_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = innatedb_df['ncbi_taxid_B'].values
b = [i for i,item in enumerate(n) if "Human" in item]
innatedb_df = innatedb_df.ix[b]

In [184]:
innatedb_df = innatedb_df[['alias_A', 'alias_B', 'pmid', 'source_database']]

In [185]:
innatedb_df.rename(columns={'alias_A': 'Protein A (gene name)', 'alias_B':'Protein B (gene name)', 'pmid':'PubMed ID', 'source_database':'Source databases'}, inplace=True)

In [186]:
# change column one to just show gene name
#i = 1
lst = []
for name in innatedb_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[-1].split(":")[1].split('(')[0])
    else:
        lst.append(np.nan)
    #i += 1
innatedb_df['Protein A (gene name)'] = lst

In [187]:
# change column two to just show gene name
#i = 1
lst = []
for name in innatedb_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name) 
    if name != 'nan':
        lst.append(name.split('|')[-1].split(":")[1].split('(')[0])
    else:
        lst.append(np.nan)
    #i += 1
innatedb_df['Protein B (gene name)'] = lst

In [188]:
# drop data for which no gene name is provited (or ensamble ID)
innatedb_df.dropna(how='any', inplace=True, axis=0)

In [189]:
lst = []

for pub in innatedb_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split(':')[1]
    lst.append(pub)
innatedb_df['PubMed ID'] = lst

In [190]:
innatedb_df.drop_duplicates(inplace=True)

In [191]:
innatedb_uf_df = innatedb_df.copy()

In [192]:
concatliketerms(innatedb_uf_df)

 Loop 82: 10800 Out of 10815 

In [193]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
innatedb_uf_df.to_csv(filename, sep='\t', index=False) 

In [194]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
innatedb_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [195]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in innatedb_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(innatedb_df['PubMed ID'].unique())), end=' ', flush=True)
    if innatedb_df[innatedb_df['PubMed ID'] == pub].shape[0] > 10:
        innatedb_df.drop(innatedb_df[innatedb_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


 3000 Out of 2984 

In [196]:
innatedb_df.shape

(6885, 4)

In [197]:
lst = []
for value in innatedb_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

innatedb_df = innatedb_df.drop(innatedb_df[lst].index)

In [198]:
innatedb_df.dropna(inplace=True)

In [199]:
concatliketerms(innatedb_df)

 Loop 68: 5400 Out of 5415 

In [200]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
innatedb_df.to_csv(filename, sep='\t', index=False) 

In [201]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
innatedb_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## DIP DATA

In [27]:
dip_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/dip20160731.txt', sep='\t', index_col=False)

In [204]:
# get only human (or mouse) data
n = dip_df['Taxid interactor A'].values
b = [i for i,item in enumerate(n) if "Homo sapiens" in item]
dip_df = dip_df.ix[b]

In [205]:
dip_df =  dip_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = dip_df['Taxid interactor B'].values
b = [i for i,item in enumerate(n) if "Homo sapiens" in item]
dip_df = dip_df.ix[b]

In [206]:
dip_df = dip_df[['ID interactor A', 'ID interactor B', 'Publication Identifier(s)', 'Source database(s)']]

In [207]:
dip_df.rename(columns={'ID interactor A': 'Protein A (gene name)', 'ID interactor B':'Protein B (gene name)', 'Publication Identifier(s)':'PubMed ID', 'Source database(s)':'Source databases'}, inplace=True)

#### mapping table to convert labels from uniprot to ncbi names

In [208]:
mapping_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Doc and Mapping/HUMAN_9606_idmapping.dat', sep='\t', index_col=False, names=['UniProtKB-AC', 'ID_type', 'ID'])

In [209]:
mapping_df = mapping_df[mapping_df['ID_type']=='Gene_Name']

In [210]:
mapping_df.set_index('UniProtKB-AC', inplace=True)

In [211]:
mapping_df = mapping_df[~mapping_df.index.duplicated(keep='first')]

In [212]:
# change column one to just show gene name
for index in dip_df.index:
    found = re.search('uniprotkb:......', dip_df.ix[index, 'Protein A (gene name)'])
    if found:
        name = found.group(0).split(':')[1]
        if name in mapping_df.index.values:
            dip_df.ix[index, 'Protein A (gene name)'] = mapping_df.ix[name, 'ID']
        else:
            dip_df.ix[index, 'Protein A (gene name)'] = np.nan
    else:
        dip_df.ix[index, 'Protein A (gene name)'] = np.nan

In [213]:
# change column two to just show gene name
for index in dip_df.index:
    found = re.search('uniprotkb:......', dip_df.ix[index, 'Protein B (gene name)'])
    if found:
        name = found.group(0).split(':')[1]
        if name in mapping_df.index.values:
            dip_df.ix[index, 'Protein B (gene name)'] = mapping_df.ix[name, 'ID']
        else:
            dip_df.ix[index, 'Protein B (gene name)'] = np.nan
    else:
        dip_df.ix[index, 'Protein B (gene name)'] = np.nan

In [214]:
lst = []

for pub in dip_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')[0].split(':')[1]
    lst.append(pub)
dip_df['PubMed ID'] = lst

In [215]:
dip_df.drop_duplicates(inplace=True)

In [216]:
dip_uf_df = dip_df.copy()

In [217]:
concatliketerms(dip_uf_df)

 Loop 2: 5500 Out of 5513 

In [218]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
dip_uf_df.to_csv(filename, sep='\t', index=False) 

In [219]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
dip_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

In [220]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in dip_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(dip_df['PubMed ID'].unique())), end=' ', flush=True)
    if dip_df[dip_df['PubMed ID'] == pub].shape[0] > 10:
        dip_df.drop(dip_df[dip_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

 2100 Out of 2047 

In [221]:
lst = []
for value in dip_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

dip_df = dip_df.drop(dip_df[lst].index)

In [222]:
dip_df.dropna(inplace=True)

In [261]:
concatliketerms(dip_df)

 Loop 9: 3900 Out of 3909 

In [202]:
#mapping_df.head()

In [262]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
dip_df.to_csv(filename, sep='\t', index=False) 

In [263]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
dip_df.to_csv(filename, sep='\t', index=False, compression='gzip') 

## mentha data

In [30]:
mentha_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/mentha_human', sep=';')

In [226]:
mentha_df = mentha_df[['Gene A', 'Gene B', 'PMID']]

In [227]:
mentha_df['Source databases'] = '(mentha)'

In [228]:
mentha_df.rename(columns={'Gene A': 'Protein A (gene name)', 'Gene B':'Protein B (gene name)', 'PMID':'PubMed ID'}, inplace=True)

In [229]:
i = 1
for index in mentha_df.index:
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(mentha_df.index)), end=' ', flush=True)
    mentha_df.ix[index, 'PubMed ID'] = '|'.join(mentha_df.ix[index, 'PubMed ID'].split(' ')[0:-1])
    i +=1

 259500 Out of 259599 

In [231]:
mentha_df.drop_duplicates(inplace=True)

In [232]:
mentha_uf_df = mentha_df.copy()

In [234]:
concatliketerms(mentha_uf_df)

 Loop 16: 258600 Out of 258615 

In [246]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mentha_uf_df.to_csv(filename, sep='\t', index=False) 

In [247]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mentha_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [237]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in mentha_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(mentha_df['PubMed ID'].unique())), end=' ', flush=True)
    if mentha_df[mentha_df['PubMed ID'] == pub].shape[0] > 10:
        mentha_df.drop(mentha_df[mentha_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1

 32800 Out of 31990 

In [238]:
lst = []
for value in mentha_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

mentha_df = mentha_df.drop(mentha_df[lst].index)

In [239]:
mentha_df.dropna(inplace=True)

In [240]:
mentha_df.shape

(49155, 4)

In [242]:
concatliketerms(mentha_df)

 Loop 6: 49100 Out of 49132 

In [243]:
mentha_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A1CF,SYNCRIP,11134005|11352648,(mentha)
1,A1CF,KHSRP,10781591,(mentha)
2,A1CF,CELF2,11577082,(mentha)
3,A1CF,TNPO2,12896982,(mentha)
4,A1CF,APOBEC1,11134005|10669759,(mentha)


In [244]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
mentha_df.to_csv(filename, sep='\t', index=False) 

In [245]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
mentha_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## IntAct Data 

In [33]:
intact_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/intact.txt', sep='\t')

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
#intact_negative_df = pd.read_csv('intact_negative.txt', sep='\t')

In [ ]:
#intact_df = pd.concat([intact_df, intact_negative_df])

In [249]:
# get only human (or mouse) data
n = intact_df['Taxid interactor A'].values
b = [i for i,item in enumerate(n) if "human" in item]
intact_df = intact_df.ix[b]

In [250]:
intact_df =  intact_df.reset_index().drop('index', axis=1)

# get only human (or mouse) data
n = intact_df['Taxid interactor B'].values
b = [i for i,item in enumerate(n) if "human" in item]
intact_df = intact_df.ix[b]

In [ ]:
#intact_df.ix[0, 'Taxid interactor B']

In [251]:
intact_df = intact_df[['Alias(es) interactor A', 'Alias(es) interactor B', 'Publication Identifier(s)', 'Source database(s)']]

In [252]:
intact_df.rename(columns={'Alias(es) interactor A': 'Protein A (gene name)', 'Alias(es) interactor B':'Protein B (gene name)', 'Publication Identifier(s)':'PubMed ID', 'Source database(s)':'Source databases'}, inplace=True)

In [253]:
# change column one to just show gene name
#i = 1
lst = []
for name in intact_df['Protein A (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
intact_df['Protein A (gene name)'] = lst

In [254]:
# change column two to just show gene name
#i = 1
lst = []
for name in intact_df['Protein B (gene name)']:
    #print('\r', i, end=' ', flush=True)
    name = str(name)
    if '(gene name)' in name:
        temp = name.split('|')
        for ID in temp:
            if '(gene name)' in ID:
                lst.append(ID.split(':')[1].split('(')[0])
                break

    else:
        lst.append(np.nan)
    
    #i += 1
intact_df['Protein B (gene name)'] = lst

In [255]:
# drop data for which no gene name is provited (or ensamble ID)
intact_df.dropna(how='any', inplace=True, axis=0)

In [256]:
lst = []

for pub in intact_df['PubMed ID']:
    pub = str(pub)
    pub = pub.split('|')
    for p in pub:
        if 'pubmed' in p:
            pub = p.split(':')[1]
    lst.append(pub)
intact_df['PubMed ID'] = lst

In [257]:
intact_df.drop_duplicates(inplace=True)

In [258]:
intact_uf_df = intact_df.copy()

In [376]:
concatliketerms(intact_uf_df)

 Loop 4: 149000 Out of 149186 

In [378]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
intact_uf_df.to_csv(filename, sep='\t', index=False) 

In [379]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
intact_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [283]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in intact_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(intact_df['PubMed ID'].unique())), end=' ', flush=True)
    if intact_df[intact_df['PubMed ID'] == pub].shape[0] > 10:
        intact_df.drop(intact_df[intact_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


 6600 Out of 6055 

In [284]:
intact_df.shape

(17171, 4)

In [285]:
lst = []
for value in intact_df['PubMed ID']:
    if 'unassigned' in value:
        lst.append(True)
    else:
        lst.append(False)

intact_df = intact_df.drop(intact_df[lst].index)

In [286]:
intact_df.dropna(inplace=True)

In [287]:
intact_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,AMPH,SYNJ1,10542231,"psi-mi:""MI:0471""(MINT)"
10,SH3GL1,SYNJ1,10542231,"psi-mi:""MI:0471""(MINT)"
11,SYNJ1,SH3GL1,10542231,"psi-mi:""MI:0471""(MINT)"
13,AMPH,DNM1,10542231,"psi-mi:""MI:0471""(MINT)"
15,SH3GL1,DNM1,10542231,"psi-mi:""MI:0471""(MINT)"


In [288]:
concatliketerms(intact_df)

 Loop 80: 14000 Out of 14770 

In [289]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
intact_df.to_csv(filename, sep='\t', index=False) 

In [290]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
intact_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## SAVI Data

In [38]:
savi_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/SNAVI.sig', sep=' ', header=None)

In [46]:
savi_df[11].unique()

array(['GAP', 'Phosphorylation', 'Binding', 'Methylation',
       'Ubiquitination', 'Dephosphorylation', 'Phosphorylaion', 'GEF',
       'Opening', 'PDE', 'Cleavage', 'Sumolation', 'Synthesis',
       'Generating', 'Pumping', 'Deubiquitination', 'Importing', 'Binidng',
       'Depolymerization'], dtype=object)

In [312]:
savi_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,SYNGAP1,Q96PV0,NaN,GEF,Cytosol,RAP1A,P62834,P62835,GTPase,Cytosol,_,GAP,15312654
1,GSK3A,P49840,NaN,Kinase,Cytosol,JUN,P05412,P05627,TF,Nucleus,_,Phosphorylation,16023596
2,GSK3A,P49840,NaN,Kinase,Cytosol,JUN,P05412,P05627,TF,Nucleus,_,Phosphorylation,1846781
3,RIPK1,Q13546,Q60855,Kinase,Cytosol,SQSTM1,Q13501,Q64337,Adapter,Cytosol,+,Binding,10356400
4,CAMK2A,Q9UQM7,P11798,Kinase,Cytosol,MAP2,P11137,P20357,Cytoskeleton,Cytosol,+,Phosphorylation,7890745


In [313]:
savi_df.shape

(2007, 13)

In [314]:
savi_df = savi_df[[0,5,12]]
savi_df.rename(columns={0:'Protein A (gene name)',5:'Protein B (gene name)',12:'PubMed ID'}, inplace=True)
savi_df['Source databases'] = '(SAVI)'

In [315]:
savi_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,SYNGAP1,RAP1A,15312654,(SAVI)
1,GSK3A,JUN,16023596,(SAVI)
2,GSK3A,JUN,1846781,(SAVI)
3,RIPK1,SQSTM1,10356400,(SAVI)
4,CAMK2A,MAP2,7890745,(SAVI)


In [316]:
# drop data for which no gene name is provited (or ensamble ID)
savi_df.dropna(how='any', inplace=True, axis=0)

In [317]:
savi_df.shape

(2007, 4)

In [318]:
savi_df.drop_duplicates(inplace=True)

In [319]:
savi_df.shape

(2007, 4)

In [320]:
savi_df.shape

(2007, 4)

In [321]:
savi_uf_df = savi_df.copy()

In [322]:
concatliketerms(savi_uf_df)

 Loop 25: 1000 Out of 1497 

In [323]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_savi_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
savi_uf_df.to_csv(filename, sep='\t', index=False) 

In [331]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_savi_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
savi_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [326]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in savi_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(savi_df['PubMed ID'].unique())), end=' ', flush=True)
    if savi_df[savi_df['PubMed ID'] == pub].shape[0] > 10:
        savi_df.drop(savi_df[savi_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


 1200 Out of 1293 

In [327]:
savi_df.dropna(inplace=True)

In [328]:
concatliketerms(savi_df)

 Loop 26: 1000 Out of 1448 

In [329]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_savi_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
savi_df.to_csv(filename, sep='\t', index=False) 

In [330]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_savi_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
savi_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## KEA Data

In [332]:
kea_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/KEA/kinase-protein_interactions.csv', sep=',', header=None)

In [333]:
kea_df.head()

,0,1,2,3,4,5
0,TK,ABL,ABL1,ABL1,10964922;11781820;12522270;15174125;17192257;1...,HPRD;PHOSPHOELM;PHOSPHOPOINT;PHOSPHOSITE;SWISS...
1,TK,ABL,ABL1,ABL2,12569093;15174125;18689816,HPRD;PHOSPHOPOINT;PHOSPHOSITE
2,TK,ABL,ABL1,BCR,10405761;11780146;12124177;12522270;15174125;1...,HPRD;PHOSPHOELM;PHOSPHOPOINT;PHOSPHOSITE
3,TK,ABL,ABL1,BTK,11598012;12445832;12573241;15174125;18689816;8...,HPRD;PHOSPHOELM;PHOSPHOPOINT;PHOSPHOSITE
4,TK,ABL,ABL1,CDK5,10896159;11113134;14757045;15174125;18689816;1...,HPRD;PHOSPHOELM;PHOSPHOPOINT;PHOSPHOSITE;SAVI;...


In [334]:
kea_ph_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Input/KEA/phosphorylation_reactions.csv', sep=',', header=None)

In [335]:
kea_ph_df.head()

,0,1,2,3,4,5
0,TK,ABL,ABL1,ABI1,11418237;12672821;15174125;17101133;18689816;8...,HPRD;MINT;PHOSPHOPOINT;PHOSPHOSITE
1,TK,ABL,ABL1,ABL1,10964922;11781820;12522270;15174125;17192257;1...,HPRD;PHOSPHOELM;PHOSPHOPOINT;PHOSPHOSITE;SWISS...
2,TK,ABL,ABL1,ABL2,12569093;15174125;18689816,HPRD;PHOSPHOPOINT;PHOSPHOSITE
3,TK,ABL,ABL1,ACAP2,17981841,NETWORKIN
4,TK,ABL,ABL1,ANXA1,15174125;2457390,PHOSPHOELM;PHOSPHOSITE


In [336]:
kea_df = pd.concat([kea_df, kea_ph_df])

In [337]:
kea_df.shape

(22478, 6)

In [338]:
kea_df.drop_duplicates(inplace=True)

In [339]:
kea_df.shape

(17401, 6)

In [340]:
kea_df.reset_index(inplace=True)
kea_df.drop('index', axis=1, inplace=True)

In [341]:
#i = 1
for index in kea_df.index:
    #print('\r', str(i)+' Out of '+ str(len(kea_df.index)), end=' ', flush=True)
    kea_df.ix[index, 4] = kea_df.ix[index,4].replace(';', '|')
    kea_df.ix[index, 5] = kea_df.ix[index,5].replace(';', '|')
    #i+=1

In [342]:
kea_df = kea_df[[2,3,4,5]]

In [343]:
kea_df.rename(columns={2: 'Protein A (gene name)', 3:'Protein B (gene name)', 4:'PubMed ID', 5:'Source databases'}, inplace=True)

In [344]:
# drop data for which no gene name is provited (or ensamble ID)
kea_df.dropna(how='any', inplace=True, axis=0)

In [345]:
kea_df.drop_duplicates(inplace=True)

In [346]:
kea_uf_df = kea_df.copy()

In [347]:
concatliketerms(kea_uf_df)

 Loop 2: 17000 Out of 17400 

In [348]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_kea_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
kea_uf_df.to_csv(filename, sep='\t', index=False) 

In [349]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_kea_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
kea_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [350]:
# drop any data that was published with more then 10 PPI's per publication
i = 1
for pub in kea_df['PubMed ID'].unique():
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(kea_df['PubMed ID'].unique())), end=' ', flush=True)
    if kea_df[kea_df['PubMed ID'] == pub].shape[0] > 10:
        kea_df.drop(kea_df[kea_df['PubMed ID'] == pub].index.tolist(), inplace=True)
    i +=1


 6400 Out of 6365 

In [351]:
kea_df.dropna(inplace=True)

In [352]:
concatliketerms(kea_df)

 Loop 2: 8000 Out of 8059 

In [353]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_kea_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
kea_df.to_csv(filename, sep='\t', index=False) 

In [354]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_kea_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
kea_df.to_csv(filename, sep='\t', index=False, compression='gzip')

### Data Tables

In [362]:
mint_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,AANAT,YWHAZ,15644438,"psi-mi:""MI:0471""(MINT)"
1,AATF,SP1,12847090,"psi-mi:""MI:0471""(MINT)"
2,AATF,MAPT,14697667,"psi-mi:""MI:0471""(MINT)"
3,AATF,RBBP8,10783144,"psi-mi:""MI:0471""(MINT)"
4,AATF,POLR2J,10783144,"psi-mi:""MI:0471""(MINT)"


In [361]:
biogrid_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A1CF,APOBEC1,10669759|11134005,"psi-mi:""MI:0463""(biogrid)|psi-mi:""MI:0463""(bio..."
1,A1CF,SYNCRIP,11134005,"psi-mi:""MI:0463""(biogrid)"
2,A1CF,KHSRP,10781591,"psi-mi:""MI:0463""(biogrid)"
3,A2M,APOE,9831625,"psi-mi:""MI:0463""(biogrid)"
4,A2M,IL10,10714547,"psi-mi:""MI:0463""(biogrid)"


In [360]:
innatedb_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A2M,IL2,9780213,MI:0974(innatedb)
1,A2M,GZMA,10666226,MI:0974(innatedb)
2,A2M,IL4,17725582,MI:0974(innatedb)
3,A2M,CXCL8,9029135,MI:0974(innatedb)
4,A2M,TGFB2,9780213,MI:0974(innatedb)


In [359]:
dip_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A2M,APP,9501253,MI:0465(dip)
1,ABCA1,SPTLC1,25170080,MI:0465(dip)
2,ABCA1,KIF11,25170080,MI:0465(dip)
3,ABCA1,FLNA,25170080,MI:0465(dip)
4,ABCC8,KCNJ11,19805355,MI:0465(dip)


In [358]:
mentha_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,A1CF,SYNCRIP,11134005|11352648,(mentha)
1,A1CF,KHSRP,10781591,(mentha)
2,A1CF,CELF2,11577082,(mentha)
3,A1CF,TNPO2,12896982,(mentha)
4,A1CF,APOBEC1,11134005|10669759,(mentha)


In [357]:
intact_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,1B,DDX58,19193793,"psi-mi:""MI:0486""(UniProt)"
1,A2M,SPACA3,2467766,"psi-mi:""MI:0469""(IntAct)"
2,AANAT,YWHAZ,15644438,"psi-mi:""MI:0471""(MINT)"
3,AATF,MAPT,14697667,"psi-mi:""MI:0471""(MINT)"
4,AATF,RBBP8,10783144,"psi-mi:""MI:0471""(MINT)"


In [356]:
savi_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,AA,PRKCA,2119631|3934001,(SAVI)|(SAVI)
7,ABL1,DOK1,12777393,(SAVI)
6,ABL1,TP53BP1,10629029,(SAVI)
5,ABL1,CDK5,11113134,(SAVI)
8,ABL1,SHC1,8112292,(SAVI)


In [355]:
kea_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,AAK1,AP1M1,11877461,PHOSPHOELM
1,AAK1,AP2M1,11877457|11877461|15174125,PHOSPHOELM|PHOSPHOSITE
2,AAK1,NUMB,18657069|15174125,PHOSPHOELM|PHOSPHOSITE
84,ABL1,ABL1,10964922|11781820|12522270|15174125|17192257|1...,HPRD|PHOSPHOELM|PHOSPHOPOINT|PHOSPHOSITE|SWISS...
83,ABL1,CBL,10829062|11997497|12475393|12522270|12604776|1...,HPRD|MINT|PHOSPHOELM|PHOSPHOPOINT|PHOSPHOSITE


## Comparison and overlap

In [59]:
data_sets= [mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df, savi_df, kea_df]
data_sets_names= ['mint', 'biogrid', 'innatedb', 'dip', 'mentha', 'intact', 'savi', 'kea']

comp_df = pd.DataFrame(columns=data_sets_names, index=data_sets_names)
comp_df.columns.name = 'Overlap (Human)'
for i in np.arange(len(data_sets)):
    for j in np.arange(len(data_sets)):
        comp = pd.concat([data_sets[i][['Aliases for A', 'Aliases for B']], data_sets[j][['Aliases for A', 'Aliases for B']]])
        comp_df.ix[data_sets_names[i], data_sets_names[j]] = comp.duplicated().sum()

# mint_df[['Aliases for A', 'Aliases for B']].duplicated().sum()
# biogrid_df[['Aliases for A', 'Aliases for B']].duplicated().sum()
comp_df

Overlap (Human),mint,biogrid,innatedb,dip,mentha,intact,savi,kea
mint,5183,2072,299,302,2761,5058,70,413
biogrid,2072,43586,1870,1482,21695,4359,305,2488
innatedb,299,1870,4754,256,1684,598,57,423
dip,302,1482,256,4463,1741,665,40,228
mentha,2761,21695,1684,1741,49501,5150,415,2051
intact,5058,4359,598,665,5150,10998,104,728
savi,70,305,57,40,415,104,1378,364
kea,413,2488,423,228,2051,728,364,8061


## Merged Network Table

In [386]:
mint_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_human_filtered_2017_02_16.tsv', sep='\t')
biogrid_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_human_filtered_2017_02_15.tsv', sep='\t')
innatedb_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_human_filtered_2017_02_15.tsv', sep='\t')
dip_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_human_filtered_2017_02_15.tsv', sep='\t')
mentha_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_human_filtered_2017_02_15.tsv', sep='\t')
intact_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_human_filtered_2017_02_16.tsv', sep='\t')
savi_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_savi_human_filtered_2017_02_16.tsv', sep='\t')
kea_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Filtered/ppi_network_kea_human_filtered_2017_02_16.tsv', sep='\t')

In [387]:
network_df = pd.concat([mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df, savi_df, kea_df])

In [388]:
#network_df.rename(columns={'Aliases for A':'Protein A (gene name)', 'Aliases for B':'Protein B (gene name)', 'Identifier of the publication':'PubMed ID'}, inplace=True)

In [389]:
network_df.sort_values(['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)

In [390]:
network_df.reset_index(inplace=True)
network_df.drop('index', axis=1, inplace=True)

In [391]:
network_df.shape

(150514, 4)

In [392]:
concatliketerms(network_df)

 Loop 9: 86000 Out of 86995 

In [393]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Filtered/ppi_network_human_filtered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False) 

In [394]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Filtered/ppi_network_human_filtered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
network_df.to_csv(filename, sep='\t', index=False, compression='gzip')

In [396]:
network_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,1B,DDX58,19193793,"psi-mi:""MI:0486""(UniProt)"
1,A1CF,APOBEC1,10669759|11134005|11134005|10669759,"psi-mi:""MI:0463""(biogrid)|psi-mi:""MI:0463""(bio..."
2,A1CF,CELF2,11577082,(mentha)
3,A1CF,KHSRP,10781591|10781591,"psi-mi:""MI:0463""(biogrid)|(mentha)"
4,A1CF,SYNCRIP,11134005|11134005|11352648,"psi-mi:""MI:0463""(biogrid)|(mentha)"


In [399]:
mint_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mint_human_unfiltered_2017_02_16.tsv', sep='\t')
biogrid_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_biogrid_human_unfiltered_2017_02_15.tsv', sep='\t')
innatedb_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_innatedb_human_unfiltered_2017_02_15.tsv', sep='\t')
dip_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_dip_human_unfiltered_2017_02_15.tsv', sep='\t')
mentha_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_mentha_human_unfiltered_2017_02_15.tsv', sep='\t')
intact_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_intact_human_unfiltered_2017_02_16.tsv', sep='\t')
savi_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_savi_human_unfiltered_2017_02_16.tsv', sep='\t')
kea_uf_df = pd.read_csv('/Users/moshesilverstein/Desktop/PPI Library/Output/IndividualResources/Unfiltered/ppi_network_kea_human_unfiltered_2017_02_16.tsv', sep='\t')

In [400]:
network_uf_df = pd.concat([mint_uf_df, biogrid_uf_df, innatedb_uf_df, dip_uf_df, mentha_uf_df, intact_uf_df, savi_uf_df, kea_uf_df])

In [401]:
network_uf_df.sort_values(['Protein A (gene name)', 'Protein B (gene name)'], inplace=True)

In [402]:
network_uf_df.reset_index(inplace=True)
network_uf_df.drop('index', axis=1, inplace=True)

In [413]:
concatliketerms(network_uf_df)

 Loop 2: 453000 Out of 453565 

In [414]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Unfiltered/ppi_network_human_unfiltered_%s.tsv'% str(datetime.date.today()).replace('-', '_')
network_uf_df.to_csv(filename, sep='\t', index=False) 

In [415]:
filename = '/Users/moshesilverstein/Desktop/PPI Library/Output/Combined/Unfiltered/ppi_network_human_unfiltered_%s.tsv.gz'% str(datetime.date.today()).replace('-', '_')
network_uf_df.to_csv(filename, sep='\t', index=False, compression='gzip')

## Merged DataFrame 

In [ ]:
# drop PPI to self for the DIP data 
for index in dip_df.index:
    if dip_df.ix[index, 'Aliases for A'] == dip_df.ix[index, 'Aliases for B']:
        dip_df.drop(index, inplace=True)

In [ ]:
df = pd.concat([mint_df, biogrid_df, innatedb_df, dip_df, mentha_df, intact_df, savi_df])

In [ ]:
df = df[['Aliases for A', 'Aliases for B']]

In [ ]:
df.shape

In [ ]:
df.drop_duplicates(['Aliases for A', 'Aliases for B'], inplace=True)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

## Gene Set Library

In [ ]:
# build library

i = 1

genelist = set(df['Aliases for A'].values.tolist() + df['Aliases for B'].values.tolist())

lib = pd.DataFrame()

for gene in genelist:
    if i % 100 == 0:
        print('\r', str(i)+' Out of '+ str(len(genelist)), end=' ', flush=True)
    lst = set(df[df.values == gene].values.flatten().tolist())
    lst.remove(gene)
    lst = list(lst)
    lst.insert(0, gene)
    if len(lst) >= 50:
        temp = pd.DataFrame(data=lst)
        lib = pd.concat([lib, temp.T])
    i += 1

In [ ]:
lib.to_csv('PPI_HUMAN.gmt', sep='\t', header=None, index=False)

In [ ]:
lib.head(20)

In [ ]:
lib.shape

## Count Proteins

In [3]:
mint_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_mint_human_filtered_2017_02_16.tsv', sep='\t')
biogrid_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_biogrid_human_filtered_2017_02_15.tsv', sep='\t')
innatedb_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_innatedb_human_filtered_2017_02_15.tsv', sep='\t')
dip_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_dip_human_filtered_2017_02_15.tsv', sep='\t')
mentha_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_mentha_human_filtered_2017_02_15.tsv', sep='\t')
intact_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_intact_human_filtered_2017_02_16.tsv', sep='\t')
savi_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_savi_human_filtered_2017_02_16.tsv', sep='\t')
kea_df = pd.read_csv('/Users/moshesilverstein/Documents/Harmonizome/PPI Library/Output/IndividualResources/Filtered/ppi_network_kea_human_filtered_2017_02_16.tsv', sep='\t')

In [4]:
mint_df.head()

,Protein A (gene name),Protein B (gene name),PubMed ID,Source databases
0,AANAT,YWHAZ,15644438,"psi-mi:""MI:0471""(MINT)"
1,AATF,MAPT,14697667,"psi-mi:""MI:0471""(MINT)"
2,AATF,POLR2J,10783144,"psi-mi:""MI:0471""(MINT)"
3,AATF,RBBP8,10783144,"psi-mi:""MI:0471""(MINT)"
4,AATF,SP1,12847090,"psi-mi:""MI:0471""(MINT)"


In [10]:
len(set(mint_df['Protein A (gene name)'].tolist()+mint_df['Protein B (gene name)'].tolist()))

2965

In [12]:
len(set(biogrid_df['Protein A (gene name)'].tolist()+biogrid_df['Protein B (gene name)'].tolist()))

7938

In [13]:
len(set(innatedb_df['Protein A (gene name)'].tolist()+innatedb_df['Protein B (gene name)'].tolist()))

2225

In [14]:
len(set(dip_df['Protein A (gene name)'].tolist()+dip_df['Protein B (gene name)'].tolist()))

2676

In [15]:
len(set(mentha_df['Protein A (gene name)'].tolist()+mentha_df['Protein B (gene name)'].tolist()))

9290

In [16]:
len(set(intact_df['Protein A (gene name)'].tolist()+intact_df['Protein B (gene name)'].tolist()))

4953

In [17]:
len(set(savi_df['Protein A (gene name)'].tolist()+savi_df['Protein B (gene name)'].tolist()))

582

In [19]:
len(set(kea_df['Protein A (gene name)'].tolist()+kea_df['Protein B (gene name)'].tolist()))

2791